In [ ]:
import numpy as np
import keras as keras

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

In [ ]:
keras.__version__

In [ ]:
np.random.seed(999)  # for reproducibility

In [ ]:
# Importing Sequential model type from Keras. This is simple a linear stack of neural network layers, 
# and it's perfect for the type of feed-forward CNN we're building in this tutorial.
from keras.models import Sequential

In [ ]:
# Import "core" layers from Keras. These are the layers that are used in almost any neural network:
from keras.layers import Dense, Dropout, Activation, Flatten

In [ ]:
# import the CNN layers from Keras. 
# These are the convolutional layers that will help us efficiently train on image data:
from keras.layers import Convolution2D, MaxPooling2D

In [ ]:
# import  utilities to transform our data later
from keras.utils import np_utils

In [ ]:
from keras.datasets import mnist
 
# Load pre-shuffled MNIST data into train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
# Understand the data sets now
# Training - X_train, y_train
# Testing  - Y_test, y_test
# Try all objects as below to understand
print(X_train) 
# Note: You will see that all the objects here are int32 type. (Reminder for later)

In [ ]:
# Now we are geting the size of each object so we can understand the train and test data better
print(X_train.shape)
# Note: X_train has 60,000 samples in training set, and the images are 28 pixels x 28 pixels each
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
# Note: The shape will print values as -> (n, width, height)

In [ ]:
# If you want to look at specfic item in the array try this
# Note:-- you will see the following gives the error as array items are 0 - 59999 > total 60000 
#      -- print(X_train[60000])
# Note:-- The following will work
#      -- print(X_train[0])
#      -- print(X_train[59999])
print(X_train[1])

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(X_train[1]) # Note: Sometimes you need to call pyplot.show() to display the image
# Try this too -- plt.imshow(X_train[0], aspect = 'auto')
plt.show()

In [ ]:
plt.imshow(X_train[1000], cmap=plt.get_cmap('gray'))
plt.show()

In [ ]:
plt.imshow(X_train[0], cmap=plt.get_cmap('gray'))
plt.show()

In [ ]:
#UNDERSTANDING showing image various ways
fig1 = plt.figure() # create a figure with the default size 

#im1 = np.random.rand(5,5)
ax1 = fig1.add_subplot(2,2,1) 
ax1.imshow(X_train[2], interpolation='none')
ax1.set_title('5 X 5')

#im2 = np.random.rand(50,50)
ax2 = fig1.add_subplot(4,4,4)
ax2.imshow(X_train[2000], interpolation='none')
ax2.set_title('100 X 100')


#im3 = np.random.rand(100,100)
ax3 = fig1.add_subplot(10,10,5) 
ax3.imshow(X_train[999], interpolation='none')
ax3.set_title('5 X 5')

plt.show()

** Now we are starting Preprocessing with KERAS **

In [ ]:
#Note There is another way to flatten the image. The training dataset is structured as a 3-dimensional array of instance, image width and image height. 
# For a multi-layer perceptron model we must reduce the images down into a vector of pixels. 
# In this case the 28×28 sized images will be 784 pixel input values.
# We can do this transform easily using the reshape() function on the NumPy array. 
# We can also reduce our memory requirements by forcing the precision of the pixel values to be 32 bit, 
# the default precision used by Keras anyway.

# flatten 28*28 images to a 784 vector for each image
num_pixels = X_train.shape[1] * X_train.shape[2]
print(num_pixels)
#X_train = X_train.reshape(X_train.shape[0], num_pixels).astype('float32')
#X_test = X_test.reshape(X_test.shape[0], num_pixels).astype('float32')

#NOTE: Above or Below Option

In [ ]:
# With Theano, you must explicitly declare a dimension for the depth of the input image. 
# COLOR - For example, a full-color image with all 3 RGB channels will have a depth of 3.
# MNIST images only have a depth of 1, but we must explicitly declare that.
# Target: transforming dataset from having shape (n, width, height) to (n, depth, width, height)

X_train = X_train.reshape(X_train.shape[0], 1, 28, 28)
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28)

In [ ]:
# Note: Confirming that we have depth included in the images.
print(X_test.shape)
print(X_test.shape[0])
print(X_test.shape[1])
print(X_test.shape[2])
print(X_test.shape[3])
# (60000, 1, 28, 28)

In [ ]:
# convert input data > data type to float32 and normalize our data values to the range [0, 1].
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [ ]:
# You will see that the X_train object is of the float32 type now
# Note:X_train[1]

In [ ]:
# Hmm... that may be problematic. 
# We should have 10 different classes, one for each digit, 
# Total items in the object
print(y_train.shape)
# Following shows 1-dimensional array of 60000 items
print(y_train)
# Let's take a look at the labels for the first 10 training samples or 1 to 11th
print(y_train[:10])
# Let's take a look at the labels from 1st to 11th
print(y_train[1:11])

In [ ]:
# PROBLEM: The y_train and y_test data are not split into 10 distinct class labels
#          both y_train and y_test data are represented as a single array with the class values.
# Fix:     Convert 1-dimensional class arrays to 10-dimensional class matrices
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)

In [ ]:
# We can see that both Y_train and y_test are two dimensional array
print(Y_train.shape)
print(Y_test.shape)

** Working on Model **

In [ ]:
#Declaring a sequential model
model = Sequential()
# Note: If you call the following you will see that there is no layer added to the model
# print(model.output_shape)

In [ ]:
# Adding a CNN layer to the model

# How to function is set for "the number of convolution filters to use"
# Param 1: [32],the number of rows in each convolution kernel, 
# Param 2: [1] nb_row: Number of rows in the convolution kernel.
# Param 3: [1] nb_col: Number of columns in the convolution kernel.
# Note: The input_shape values are filled from (depth, width, height) >> print X_train.shape

model.add(Convolution2D(32, 1, 1, activation='relu', input_shape=(1,28,28)))
# Note: The input shape parameter should be the shape of 1 sample. In this case, 
#       it's the same (1, 28, 28) that corresponds to  the (depth, width, height) of each digit image.
print(model.output_shape)

In [ ]:
model.add(Convolution2D(32, 1, 1, activation='relu'))
model.add(MaxPooling2D(pool_size=(1,1)))
model.add(Dropout(0.25))

In [ ]:
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

### Adding epoch to model fit
#### larger the number longer it will take to train
#### larger the number, the better the model will be

In [ ]:
epoch_count = 2

In [ ]:
model.fit(X_train, Y_train, 
          batch_size=32, nb_epoch=epoch_count, verbose=1)

In [ ]:
print(model)

In [ ]:
score = model.evaluate(X_test, Y_test, verbose=1)

In [ ]:
print(score)
print("Baseline Error: %.2f%%" % (100-score[1]*100))

In [ ]:
print(Y_train[0])
print(y_train[0])
print(X_train[0].shape)


In [ ]:
X_train

In [ ]:
X_1 = X_train[100].reshape(X_train[0].shape[0], 1, 28, 28)

In [ ]:
pred1 = model.predict(X_1,batch_size=32, verbose=1)
print(pred1)
print(pred1[0])
xbar = range(len(pred1[0]))
print(xbar)
plt.bar(xbar,pred1[0])
print(np.argmax(pred1[0]))
plt.show()

In [ ]:
print(X_1.shape)
print(y_train[0])

In [ ]:
Y_test[0]

In [ ]:
label1 = np.argmax(pred1[0])
print(label1)

In [ ]:
X_2 = X_train[1].reshape(X_train[1].shape[0], 1, 28, 28)
print(X_2.shape)

In [ ]:
pred2 = model.predict(X_2,batch_size=32, verbose=1)
print(pred2[0])
print(np.argmax(pred2[0]))